In [3]:
import jax.numpy as jnp
import numpy as np
h_size = 5
head = 4
r1 = jnp.arange(2)
r0 = jnp.arange(2)


In [4]:
r0[:, None] * (1 - (jnp.arange(h_size) / (h_size - 1)))[None, :] + 0.1 * ((jnp.arange(h_size) + 1) % 3)


Array([[0.1 , 0.2 , 0.  , 0.1 , 0.2 ],
       [1.1 , 0.95, 0.5 , 0.35, 0.2 ]], dtype=float32)

In [16]:
r0 = jnp.arange(2)
r0[:, None] * (1 - (jnp.arange(D) / (D - 1)))[None, :] + 0.1 * ((jnp.arange(D) + 1) % 3)

Array([[0.1, 0.2, 0. ],
       [1.1, 0.7, 0. ]], dtype=float32)

In [10]:
import torch
import torch.nn as nn
m = nn.ZeroPad2d(2)
input = torch.randn(1, 1, 3, 3)
input
m(input)
# using different paddings for different sides
m = nn.ZeroPad2d((0, 0, 1, -1))
m(input)

tensor([[[[ 0.0000,  0.0000,  0.0000],
          [ 1.1325, -0.5770,  0.4798],
          [-0.2277, -0.3857, -0.1964]]]])

In [18]:
import jax
a1, a2, a3 = jnp.ones((3, 4))

In [19]:
a1

Array([1., 1., 1., 1.], dtype=float32)

In [20]:
x = jnp.ones(3)


In [23]:
jnp.ones((3, 4)) + jnp.ones(3)[:, None]

Array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]], dtype=float32)

In [30]:
a = jnp.ones(3)
b = jnp.ones((3, 2, 4))
jnp.bmm(a, b)

AttributeError: module 'jax.numpy' has no attribute 'bmm'